In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn import metrics

In [2]:
#Reading dataset
df=pd.read_csv('RDKit.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Columns: 224 entries, Cate_name to Yield
dtypes: float64(177), int64(45), object(2)
memory usage: 119.1+ KB


In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_MaxAbsEStateIndex  Cate_MaxEStateIndex  Cate_MinAbsEStateIndex  \
0                8.669259             8.669259                0.076389   
1                8.669259             8.669259                0.076389   
2                8.669259             8.669259                0.076389   
3                8.669259             8.669259                0.076389   
4                8.850093             8.850093                0.060185   

   Cate_MinEStateIndex  Cate_qed  Cate_SPS  Cate_MolWt  Cate_HeavyAtomMolWt  \
0            -0.076389  0.490728  9.000000     110.112              104.064   
1            -0.076389  0.490728  9.000000     110.112              104.064   
2            -0.076389  0.490728  9.000000     110.112              104.064   
3            -0.076389  0.490728  9.000000     110.112              104.064   
4            -0.068889  0.513122  9.444444     124.139              116.075   

   Cate_ExactMolWt  Cate_NumValenceElectrons  ...  P_NumAromat

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
best_model_parametors = []
data_cv_score = []
loo_score = []

#knn
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    #autoscaling
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_train = a_X_train.dropna(how='any', axis=1)
    a_X_test = a_X_test[a_X_train.columns]
    
    param_knn={'n_neighbors':[1,2,3,4,5,6,7,8,9,10], 'p':[1,2,3,4,5]}
    reg_knn = GridSearchCV(KNeighborsRegressor(), param_grid=param_knn, cv=5, n_jobs=2)
    reg_knn.fit(a_X_train,y_train['Yield'])
    reg_best = reg_knn.best_estimator_
    y_pred1 = reg_best.predict(a_X_train)
    y_pred2 = reg_best.predict(a_X_test)

    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)

    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)
    
    parametors = reg_knn.best_params_
    best_model_parametors.append(parametors)
    cv_score = reg_knn.best_score_
    data_cv_score.append(cv_score)
    
    print('----------------------')
    print('seed:', seed)
    print("Best Model Parameter:",reg_knn.best_params_)
    print("Best Model Score:",reg_knn.best_score_)
    print('R2_test:', r2_test)
print('R2_train_means:', sum(data_r2_train)/10)
print('CV_score_means:', sum(data_cv_score)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.7696875405146143
R2_test: 0.6150009708636872
----------------------
seed: 1
Best Model Parameter: {'n_neighbors': 8, 'p': 1}
Best Model Score: 0.7181073744426569
R2_test: 0.7241684771287875
----------------------
seed: 2
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.6768916547330088
R2_test: 0.7066700822715257
----------------------
seed: 3
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.642959792480353
R2_test: 0.733016711580107
----------------------
seed: 4
Best Model Parameter: {'n_neighbors': 7, 'p': 1}
Best Model Score: 0.7085482428912784
R2_test: 0.7169245711019894
----------------------
seed: 5
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.7357842656198377
R2_test: 0.6725717478393922
----------------------
seed: 6
Best Model Parameter: {'n_neighbors': 6, 'p': 1}
Best Model Score: 0.6873846290332973
R2_test: 0.7235

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])
data_cv_score_pd = pd.DataFrame(data=data_cv_score, columns=['cv_score'])
data_parametors = pd.DataFrame([best_model_parametors])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd, data_cv_score_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/RDKit/knn_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test  \
0     0.793725   12.332039   8.574468  0.615001  22.440132  15.801587   
1     0.774224   14.288849   9.763298  0.724168  16.347441  11.648810   
2     0.762468   14.823648   9.322695  0.706670  16.842446  14.341270   
3     0.755539   15.100043   9.981763  0.733017  16.007378  11.517007   
4     0.746570   14.781121  10.343465  0.716925  17.349380  11.210884   
5     0.772643   13.568319   9.425532  0.672572  19.955440  13.746032   
6     0.750226   14.317223  10.102837  0.723563  18.239551  11.944444   
7     0.801167   13.790684   9.357447  0.665151  17.422481  14.380952   
8     0.776834   14.014709   9.249240  0.683371  17.689087  13.006803   
9     0.788933   14.625473  10.911854  0.707494  14.356762  10.748299   
mean  0.772233   14.164211   9.703260  0.694793  17.665010  12.834609   
std   0.016842    0.727183   0.597970  0.032771   2.014534   1.515024   

      cv_score  
0     0.769688  
1     0.718107  